In [1]:
import csv
with open('basket.csv', 'r', encoding='utf-8') as cf:
    transactions = []
    r = csv.reader(cf)
    for row in r:
        transactions.append(row)

In [2]:
transactions

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['콜라', '맥주', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [5]:
from apyori import apriori
rules = apriori(transactions, min_support=0.1, min_confidence=0.1)
results = list(rules)

In [4]:
!pip install apyori

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=1bec89f324f803eb3e7cf295bf92417fe1efa5eb45bd0a191554246d15cda8b6
  Stored in directory: c:\users\kosa\appdata\local\pip\cache\wheels\77\3d\a6\d317a6fb32be58a602b1e8c6b5d6f31f79322da554cad2a5ea
Successfully built apyori


In [6]:
type(results)

list

In [7]:
results[0]

RelationRecord(items=frozenset({'맥주'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주'}), confidence=0.4, lift=1.0)])

In [12]:
import pandas as pd
result_df = pd.DataFrame(None, columns=["lhs", "rhs", "support", "confidence", "lift"])
index=0
for result in results:
    support = result[1]
    for ordered_item in result[2]:
        lhs = ','.join(ordered_item[0])
        rhs = ','.join(ordered_item[1])
        confidence = ordered_item[2]
        lift = ordered_item[3]
        result_df.loc[index] = [lhs, rhs, support, confidence, lift]
        index += 1

In [13]:
result_df.head(25)

,lhs,rhs,support,confidence,lift
0,,맥주,0.4,0.400000,1.000000
1,,소주,0.6,0.600000,1.000000
2,,오렌지주스,0.4,0.400000,1.000000
3,,와인,0.6,0.600000,1.000000
4,,콜라,0.8,0.800000,1.000000
5,,"맥주,소주",0.2,0.200000,1.000000
6,맥주,소주,0.2,0.500000,0.833333
7,소주,맥주,0.2,0.333333,0.833333
8,,"와인,맥주",0.2,0.200000,1.000000
9,맥주,와인,0.2,0.500000,0.833333


In [14]:
result_df.loc[(result_df.lhs=="오렌지주스") & (result_df.rhs=="와인")]

,lhs,rhs,support,confidence,lift
24,오렌지주스,와인,0.2,0.5,0.833333


In [23]:
result_df.loc[(result_df.lhs=="맥주") & (result_df.rhs=="와인")]

,lhs,rhs,support,confidence,lift
9,맥주,와인,0.2,0.5,0.833333


In [1]:
import requests
rss_url = "http://fs.jtbc.joins.com/RSS/economy.xml"
jtbc_economy = requests.get(rss_url)

In [2]:
from bs4 import BeautifulSoup
economy_news_list = BeautifulSoup(jtbc_economy.content, "xml")
link_list = economy_news_list.select("item > link")

In [3]:
len(link_list)

20

In [4]:
link_list[0].text

'https://news.jtbc.co.kr/article/article.aspx?news_id=NB12198470'

In [5]:
!pip install konlpy

In [29]:
!pip install JPype1-1.1.2-cp38-cp38-win_amd64.whl

ERROR: JPype1-1.1.2-cp38-cp38-win_amd64.whl is not a supported wheel on this platform.


In [6]:
from konlpy.tag import Kkma
kkma = Kkma()

In [7]:
stopwords = {"앵커", "기자", "기업", "영상", "영상디자인", "디자인"}
news = [] #transaction
for link in link_list:
    news_url = link.text
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.content,"html.parser")
    news_content = news_soup.select_one("#articlebody > .article_content")
    news_nouns = kkma.nouns(news_content.text)
    item_list = list(filter(lambda word: len(word)>1, news_nouns))
    item_list = [t for t in item_list if t not in stopwords]

    news.append(item_list)

In [10]:
import pandas as pd
result_df = pd.DataFrame(None, columns=["lhs","rhs","support","confidence","lift"])
index=0
for row in results:
    support = row[1]
    ordered_stat = row[2]

    for ordered_item in ordered_stat:
        lhs = " ".join(ordered_item[0])
        rhs = " ".join(ordered_item[1])
        confidence = ordered_item[2]
        lift = ordered_item[3]
        result_df.loc[index] = [lhs, rhs, support, confidence, lift]
        index = index + 1

In [11]:
pd.options.display.max_rows = 1000

In [12]:
result_df.loc[result_df.lhs==""].sort_values(by=["support"], ascending=False)

,lhs,rhs,support,confidence,lift
17,,정부,0.50,0.50,1.0
14,,올해,0.45,0.45,1.0
0,,10,0.40,0.40,1.0
13,,오늘,0.40,0.40,1.0
9,,때문,0.35,0.35,1.0
2,,20,0.35,0.35,1.0
15,,운영,0.35,0.35,1.0
12,,서울,0.35,0.35,1.0
10,,문제,0.30,0.30,1.0
1,,1년,0.30,0.30,1.0


In [16]:
find_condition = result_df.lhs.str.contains("직구")
result_df.loc[find_condition].sort_values(by=["lift"], ascending=False)

,lhs,rhs,support,confidence,lift
184,정부 직구,국내 규제,0.25,1.0,4.000000
169,직구 인증,국내 규제,0.25,1.0,4.000000
172,인증 직구 국내,규제,0.25,1.0,4.000000
237,직구 국내 규제,정부 인증,0.25,1.0,4.000000
178,직구,정부 국내 규제,0.25,1.0,4.000000
181,직구 국내,정부 규제,0.25,1.0,4.000000
183,직구 규제,정부 국내,0.25,1.0,4.000000
55,직구,규제,0.25,1.0,4.000000
234,정부 직구,인증 국내 규제,0.25,1.0,4.000000
187,정부 직구 국내,규제,0.25,1.0,4.000000


In [9]:
from apyori import apriori
rules = apriori(news, min_support=0.25, min_confidence=0.2)
results = list(rules)

In [18]:
result_df.loc[(result_df.lhs.str.contains("직구")) & (result_df.rhs=="규제")].sort_values(by=["support"], ascending=False)

,lhs,rhs,support,confidence,lift
55,직구,규제,0.25,1.0,4.0
93,직구 국내,규제,0.25,1.0,4.0
129,직구 인증,규제,0.25,1.0,4.0
136,정부 직구,규제,0.25,1.0,4.0
172,인증 직구 국내,규제,0.25,1.0,4.0
187,정부 직구 국내,규제,0.25,1.0,4.0
218,정부 직구 인증,규제,0.25,1.0,4.0
248,인증 직구 국내 정부,규제,0.25,1.0,4.0
